# RAG mit Langchain

## Einführung

Im folgenden wird eine RAG für die Magpie gebaut. Ziel ist es, einen Chatbot zu genieren, der Fragen in natürlicher Sprache aufnimmt, diese in passende SQL-Abfragen umwandelt. Diese werden wiederum dem hinter dem Chatbot stehenden LMM als Kontext übermittelt, sodass dieser wiederum in natürlicher Sprache antworten kann. 

In [3]:
import os

# Aktuelles Arbeitsverzeichnis ermitteln
os.getcwd()
os.chdir("c:/Users/Hueck/OneDrive/Dokumente/GitHub/magpie_langchain")

## Laden der Magpie

Wir laden weiterhin die Magpie und stellen eine Verbindung zu ihr her:

In [4]:
import duckdb

conn = duckdb.connect("data/magpie.db")
cursor = conn.cursor()
#conn.close() 

## Exploration der Magpie

Zu exploration wählen wir den Datensatz `datensatz_drittmittel_hochschule` und wandeln diesen in einen pandas-dataframe um:

In [5]:
import pandas as pd

# Tabelle 'datensatz_fue_erhebung' in ein Pandas-DataFrame laden
query = "SELECT * FROM datensatz_drittmittel_hochschule;"
df = pd.read_sql(query, conn)  # conn ist die Verbindung zu deiner DuckDB
 
# DataFrame anzeigen
print(df)
conn.close()

C:\Users\Hueck\AppData\Local\Temp\ipykernel_6304\1784361715.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)  # conn ist die Verbindung zu deiner DuckDB


       jahr      id                 Variable       Zeit  \
0      2006   30746     Drittmittel vom Bund 2006-01-01   
1      2007   30747     Drittmittel vom Bund 2007-01-01   
2      2008   30748     Drittmittel vom Bund 2008-01-01   
3      2010   30750     Drittmittel vom Bund 2010-01-01   
4      2011   30751     Drittmittel vom Bund 2011-01-01   
...     ...     ...                      ...        ...   
87855  2021  118392  Drittmittel von der DFG 2021-01-01   
87856  2021  118406  Drittmittel von der DFG 2021-01-01   
87857  2021  118418  Drittmittel von der DFG 2021-01-01   
87858  2021  118475  Drittmittel von der DFG 2021-01-01   
87859  2021  118595  Drittmittel von der DFG 2021-01-01   

                                              Hochschule  \
0                                     Universität Kassel   
1                                     Universität Kassel   
2                                     Universität Kassel   
3                                     Universität K

## Laden von llama3.1 über Ollama

In einem Schritt wird über Ollama das LLM `llama3.1` geladen:

In [6]:
from langchain_ollama import ChatOllama
import re

llm = ChatOllama(
    model="llama3.1:8b-instruct-q4_0",
    temperature=0,
    server_url="http://127.0.0.1:11434",
)

Im Folgenden wird zunächst die `SQLDatabase`-Klasse aus dem Modul `langchain_community.utilities` importiert. Anschließend wird mit `SQLDatabase.from_uri("duckdb:///data/drittmittel_hs.db")` eine Verbindung zur DuckDB-Datenbank namens `drittmittel_hs.db` im Verzeichnis `data` aufgebaut.

In [7]:
from langchain_community.utilities import SQLDatabase
db = SQLDatabase.from_uri("duckdb:///data/drittmittel_hs.db")

c:\Users\Hueck\miniconda3\envs\RAG_LLM\lib\site-packages\duckdb_engine\__init__.py:174: DuckDBEngineWarning: duckdb-engine doesn't yet support reflection on indices
  warnings.warn(


In [8]:
def get_unique_values_for_variables(db, dataset, variables):
    results = {}
    for var in variables:
        query = f"SELECT DISTINCT {var} FROM {dataset}"
        unique_values = [row.strip() for row in db.run(query).split("\n") if row.strip()]
        results[var] = unique_values
    return results
# Beispielanwendung:
ergebnis = get_unique_values_for_variables(db, "datensatz_drittmittel_hochschule", ["Hochschule", "Variable"])

Im folgenden Code werden zunächst die notwendigen Module und Klassen importiert, um SQL-Abfragen basierend auf natürlicher Sprache zu erstellen, auszuführen und die Ergebnisse an das LMM zurückzugeben. 

- Zuerst definieren wir die Funktion `get_sql_chain`, in der eine Verarbeitungskette erstellt wird. Diese Kette generiert basierend auf einer Eingabe und der Datenbankschema-Information eine SQL-Abfrage. Das zugrundeliegende Template gibt dabei vor, wie diese Abfragen strukturiert sein sollen, um relevante Daten effizient abzurufen.
- Darauf aufbauend implementiert die Funktion `natural_language_chain` die vollständige Verarbeitung: Zunächst wird die Datenbankschema-Information ausgelesen und eine SQL-Abfrage für die Eingabefrage generiert. Diese Abfrage wird anschließend gegen die Datenbank ausgeführt. Mithilfe einer weiteren Vorlage wird das SQL-Ergebnis in eine präzise Antwort in natürlicher Sprache umgewandelt.

In [9]:
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
from langchain_core.output_parsers import StrOutputParser
from langchain.chains import create_sql_query_chain


def get_sql_chain(llm, db, table_info, top_k=10):
    template = f"""Given an input question, first create a syntactically
    correct SQL query to run in {db.dialect}, then look at the results of the
    query and return the answer to the input question. You can order the
    results to return the most informative data in the database.
    
    Unless otherwise specified, do not return more than {{top_k}} rows.

    Never query for all columns from a table. You must query only the
    columns that are needed to answer the question. Wrap each column name
    in double quotes (") to denote them as delimited identifiers.

    Pay attention to use only the column names present in the tables
    below. Be careful to not query for columns that do not exist. Also, pay
    attention to which column is in which table. Query only the columns you
    need to answer the question.
 
    Please carefully think before you answer.

    Here is the schema for the database:
    {{table_info}}

    Additional info: {{input}}

    Return only the SQL query such that your response could be copied
    verbatim into the SQL terminal.
    """

    prompt = PromptTemplate.from_template(template)

    sql_chain = create_sql_query_chain(llm, db, prompt)

    return sql_chain

def natural_language_chain(question, llm, db):
    table_info = db.get_table_info()
    sql_chain = get_sql_chain(llm, db, table_info=table_info)

    template = f"""
        You are a chatbot named >>PortaBot<< created by the 
        >>Stifterverband für die Deutsche Wissenschaft<<. Based on the table schema given below, the SQL query and the SQL response, enter an answer
        that corresponds to the language of the user's question. Think carefully and make sure that your answer is precise and easy to understand.

        SQL Query: {{query}}
        User question: {{question}}
        SQL Response: {{response}}
        """

    prompt = PromptTemplate.from_template(template)

    # Create the intermediate chain to extract SQL query
    intermediate_chain = RunnablePassthrough.assign(query=sql_chain)

    # Get the SQL query
    intermediate_result = intermediate_chain.invoke({"question": question})
    sql_query = intermediate_result["query"]

    # Debug: Print the SQL query
    print("Generated SQL Query for Debugging:")
    print(sql_query)

    # Continue with the full chain execution
    chain = (
        intermediate_chain.assign(
            response=itemgetter("query") | QuerySQLDataBaseTool(db=db)
        )
        | prompt
        | llm
        | StrOutputParser()
    )

    response = chain.invoke({"question": question})

    print(response)

    return response

In [10]:
_ = natural_language_chain('Wie hoch waren die "Drittmittel von Gemeinden und Zweckverbänden" der Universität Kassel im Jahr 2008?', llm, db)

Generated SQL Query for Debugging:
SELECT "Wert"
FROM datensatz_drittmittel_hochschule
WHERE "Variable" = 'Drittmittel von Gemeinden und Zweckverbänden'
AND "Hochschule" = 'Universität Kassel'
AND jahr = 2008;
Die Drittmittel von Gemeinden und Zweckverbänden der Universität Kassel beliefen sich im Jahr 2008 auf 107.659,99 Euro.


In [ ]:
_ = natural_language_chain('Was ist der Stifterverband?', llm, db)
